In [1]:
import os, sqlite3
import pandas as pd
import numpy as np
from zoneinfo import ZoneInfo
from google.protobuf.json_format import MessageToDict
# import sys; sys.path.insert(0, "gen")
from gen import messages_pb2  # generated by setup.sh

# Eye-Tracking

In [2]:
data_path = "/store/kruu/eye_tracking/training_data/010/Scenario 3/ET/010_scenario_3_gaze_data_fusion.tsv"
df_eye_tracking = pd.read_csv(data_path, sep="\t")

/tmp/ipykernel_1817408/2446824208.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eye_tracking = pd.read_csv(data_path, sep="\t")


In [3]:
def slice_between_events(df, start="ScreenRecordingStart", end="ScreenRecordingEnd", *, include_bounds=False):
    s = df["Event"]

    starts = np.flatnonzero(s.eq(start))
    ends   = np.flatnonzero(s.eq(end))

    if len(starts) != 1 or len(ends) != 1:
        raise ValueError(f"Expected exactly one '{start}' and one '{end}' "
                         f"(got {len(starts)} and {len(ends)}).")
    i, j = int(starts[0]), int(ends[0])
    if j <= i:
        raise ValueError(f"'{end}' occurs before '{start}' (positions {j} <= {i}).")

    if include_bounds:
        return df.iloc[i:j+1]     # includes the start & end rows
    else:
        return df.iloc[i+1:j]     # strictly between them
    
df_eye_tracking_sliced = slice_between_events(df_eye_tracking, include_bounds=True)
df_eye_tracking_sliced = df_eye_tracking_sliced.drop(columns=["Mouse position X [DACS px]", "Mouse position Y [DACS px]"])

In [4]:
date_str = df_eye_tracking_sliced["Recording date"].astype(str).str.replace(r"\.$", "", regex=True)
base_start = pd.to_datetime(
    date_str + " " + df_eye_tracking_sliced["Recording start time"].astype(str),
    format="%d.%m.%Y %H:%M:%S.%f",
    errors="coerce",
)

offset = pd.to_timedelta(pd.to_numeric(df_eye_tracking_sliced["Recording timestamp [ms]"], errors="coerce"), unit="ms")
df_eye_tracking_sliced["ts_cet"] = base_start + offset

cet = df_eye_tracking_sliced["ts_cet"].dt.tz_localize(
    "Europe/Zurich", ambiguous="infer", nonexistent="shift_forward"
)

df_eye_tracking_sliced["epoch_ms"] = (cet.view("int64") // 1_000_000).astype("Int64")

/tmp/ipykernel_1817408/2605383359.py:15: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df_eye_tracking_sliced["epoch_ms"] = (cet.view("int64") // 1_000_000).astype("Int64")


# Events db for mouse tracking

In [5]:
TZ = ZoneInfo("Europe/Zagreb")

def load_mouse_positions(db_path, start_epoch_ms, end_epoch_ms, limit=None, batch=20_000):
    """Return rows between UNIX epochs in ms with only ts_cet and mouse position."""

    # Read-only, immutable: faster & avoids locks
    con = sqlite3.connect(f"file:{db_path}?mode=ro&immutable=1", uri=True)
    con.text_factory = bytes
    # Read-only speed PRAGMAs (best-effort; safe if ignored)
    con.execute("PRAGMA query_only=ON")
    con.execute("PRAGMA mmap_size=268435456")        # 256 MiB
    con.execute("PRAGMA temp_store=MEMORY")

    sql = ('SELECT id, epoch_ms, payload FROM "events" '
           'WHERE epoch_ms BETWEEN ? AND ? '
           'ORDER BY epoch_ms')
    cur = con.execute(sql, (start_epoch_ms, end_epoch_ms))

    ids, epochs, xs, ys = [], [], [], []
    seen = 0
    while True:
        rows = cur.fetchmany(batch)
        if not rows:
            break
        for id_, ms, blob in rows:
            ev = messages_pb2.Event()
            ev.ParseFromString(blob)  # decode protobuf

            # Extract just what you need; be defensive about presence/names
            mp = getattr(getattr(ev, "asd_event", None), "mouse_position", None)
            x = getattr(mp, "x", None) or getattr(mp, "pos_x", None) if mp is not None else None
            y = getattr(mp, "y", None) or getattr(mp, "pos_y", None) if mp is not None else None

            ids.append(int(id_)); epochs.append(int(ms)); xs.append(x); ys.append(y)
            seen += 1
            if limit is not None and seen >= int(limit):
                rows = []  # to break outer loop
                break

        if limit is not None and seen >= int(limit):
            break

    con.close()

    df = pd.DataFrame(
        {"id": ids, "epoch_ms": epochs, "Mouse position X": xs, "Mouse position Y": ys}
    )
    if not df.empty:
        ts_utc = pd.to_datetime(df["epoch_ms"].astype("int64"), unit="ms", utc=True)
        df["ts_cet"] = ts_utc.dt.tz_convert(TZ)
        # put ts_cet first
        df = df[["ts_cet", "epoch_ms", "Mouse position X", "Mouse position Y", "id"]]

    return df


In [6]:
DB = "/store/kruu/eye_tracking/training_data/010/Scenario 3/simulator/010_scenario_3/polaris-events-2025-09-04T15_04_14.db"
df_mouse = load_mouse_positions(DB, int(df_eye_tracking_sliced.epoch_ms.min()), int(df_eye_tracking_sliced.epoch_ms.max()))

# Merge ET + mouse: Full Union time

In [7]:
TZ = ZoneInfo("Europe/Zagreb")

dfe = df_eye_tracking_sliced.sort_values("epoch_ms").astype({"epoch_ms":"int64"})
dfm = df_mouse.sort_values("epoch_ms").astype({"epoch_ms":"int64"})

# 1) union of all timestamps
timeline = pd.DataFrame({"epoch_ms": np.union1d(dfe["epoch_ms"].values, dfm["epoch_ms"].values)})

# 2) attach eye-tracking columns by nearest (or exact if you set TOL=0)
TOL = 0  # ms
timeline = pd.merge_asof(
    timeline, dfe,
    on="epoch_ms",
    direction="nearest",
    tolerance=TOL
)

# 3) attach mouse columns by nearest (same tolerance)
timeline = pd.merge_asof(
    timeline,
    dfm[["epoch_ms", "Mouse position X", "Mouse position Y"]],
    on="epoch_ms",
    direction="nearest",
    tolerance=TOL
)

timeline["ts_utc"] = pd.to_datetime(timeline["epoch_ms"].astype("int64"), unit="ms", utc=True)
timeline["ts_cet"] = timeline["ts_utc"].dt.tz_convert(TZ)


In [8]:
timeline.columns

Index(['epoch_ms', 'Recording timestamp [ms]', 'Computer timestamp [ms]',
       'Sensor', 'Participant name', 'Recording date', 'Recording start time',
       'Timeline name', 'Event', 'Event value', 'Gaze point X [DACS px]',
       'Gaze point Y [DACS px]', 'Gaze point left X [DACS px]',
       'Gaze point left Y [DACS px]', 'Gaze point right X [DACS px]',
       'Gaze point right Y [DACS px]', 'Gaze direction left X [DACS norm]',
       'Gaze direction left Y [DACS norm]',
       'Gaze direction left Z [DACS norm]',
       'Gaze direction right X [DACS norm]',
       'Gaze direction right Y [DACS norm]',
       'Gaze direction right Z [DACS norm]', 'Pupil diameter left [mm]',
       'Pupil diameter right [mm]', 'Pupil diameter filtered [mm]',
       'Eye openness left [mm]', 'Eye openness right [mm]',
       'Eye openness filtered [mm]', 'Gaze point left X [DACS mm]',
       'Gaze point left Y [DACS mm]', 'Gaze point right X [DACS mm]',
       'Gaze point right Y [DACS mm]', 'Eye mo

In [9]:
timeline.to_parquet("/store/kruu/eye_tracking/training_data/010/Scenario 3/taskRecognition/raw_inputs.parquet")